In [ ]:
import pandas as pd
import numpy as np

In [ ]:
path = ### PATH TO DATA HERE ###

In [ ]:
cog_cat = pd.read_csv(path + 'Cognitive_Categorization.csv')
dod = pd.read_csv(path + 'Conclusion_of_Study_Participation.csv')
dod2 = pd.read_csv(path + 'FOUND_Enrollment_Status.csv')

In [ ]:
pd_on = pd.read_csv('pd_on.csv')

In [ ]:
outcome_df = pd_on[['PATNO', 'EVENT_ID', 'INFODT']]

In [ ]:
outcome_df['Dyskinesia'] = (pd_on['NP4WDYSK'] > 2) & (pd_on['NP4DYSKI'] > 2)
outcome_df['MotorFluctuation'] = (pd_on['NP4OFF'] > 2)
outcome_df['MotorImpairment'] = (pd_on['NP4FLCTI'] > 2)
outcome_df['HY'] = (pd_on['NHY'] > 3)

In [ ]:
cog_df = pd_on[['PATNO', 'EVENT_ID', 'INFODT']].merge(cog_cat, how='left', on=['PATNO', 'EVENT_ID', 'INFODT'])

In [ ]:
dementia_store = pd.DataFrame(columns=['PATNO', 'EVENT_ID', 'INFODT'])

pids = cog_df[cog_df.COGSTATE==3.0].PATNO.unique()
for pid in pids:
    rec = cog_df[cog_df.PATNO==pid]
    if (rec.COGSTATE==3.0).sum() > 1:
        dt = pd.to_datetime(rec[rec.COGSTATE == 3].INFODT).values
        time_delta = (dt - dt[0])/pd.Timedelta(1,'D')/365
        idx = rec[rec.COGSTATE == 3].INFODT.index
        if (time_delta < 0).any():
            print('Flipped')
            time_delta = time_delta + abs(np.min(time_delta))
        if (time_delta > 0.9).any():
            dementia_store = dementia_store.append(cog_df.iloc[idx.values[time_delta > 0.9]][['PATNO', 'EVENT_ID', 'INFODT']], ignore_index=True)

In [ ]:
dementia_store['Dementia'] = True

In [ ]:
mci_store = pd.DataFrame(columns=['PATNO', 'EVENT_ID', 'INFODT'])

pids = cog_df[cog_df.COGSTATE==2.0].PATNO.unique()
for pid in pids:
    rec = cog_df[cog_df.PATNO==pid]
    if (rec.COGSTATE==2.0).sum() > 1:
        dt = pd.to_datetime(rec[rec.COGSTATE == 2.0].INFODT).values
        time_delta = (dt - dt[0])/pd.Timedelta(1,'D')/365
        idx = rec[rec.COGSTATE == 2.0].INFODT.index
        if (time_delta < 0).any():
            print('Flipped')
            time_delta = time_delta + abs(np.min(time_delta))
        if (time_delta > 0.9).any():
            mci_store = mci_store.append(cog_df.iloc[idx.values[time_delta > 0.9]][['PATNO', 'EVENT_ID', 'INFODT']], ignore_index=True)

In [ ]:
mci_store['MCI'] = True

In [ ]:
outcome_df = outcome_df.merge(dementia_store, how='left', on=['PATNO', 'EVENT_ID', 'INFODT'])
outcome_df = outcome_df.merge(mci_store, how='left', on=['PATNO', 'EVENT_ID', 'INFODT'])

In [ ]:
outcome_df.loc[outcome_df.Dementia.isnull(), 'Dementia'] = False
outcome_df.loc[outcome_df.MCI.isnull(), 'MCI'] = False

In [ ]:
dod_store = dod[(dod.WDRSN==6) & (dod.PATNO.isin(pd_on.PATNO.unique()))][['PATNO', 'INFODT']]

In [ ]:
for pid in dod2[(dod2.FOSTATUS==9) & (dod2.PATNO.isin(pd_on.PATNO.unique()))].PATNO:
    if ~dod_store.PATNO.isin([pid]).any():
        dod_store = dod_store.append(pd.DataFrame({'PATNO': pid, 'INFODT': 'NaN'}, index=[0]), ignore_index=True)

In [ ]:
outcome_df['Death'] = False

In [ ]:
for pid in dod_store.PATNO:
    last_idx = np.argsort(pd.to_datetime(outcome_df[outcome_df.PATNO==pid].INFODT)).index[-1]
    outcome_df.at[last_idx, 'Death'] = True

In [ ]:
outcome_df.to_csv('Outcome.csv')